# Genex Research Agents — Multi-Model Therapy Research + Final Parent Summary (ADK Notebook)

## What this notebook is
This notebook is a **Genex “research agent” prototype** built with **Google ADK (Agent Development Kit)**. It answers parent questions (e.g., “What therapies help?”) by running a **multi-model research pass** (Gemini + GPT + Claude, when keys are available) and then producing **one merged, parent-friendly final summary**.

Core idea:
- **Profile → Parallel research → Aggregation**
- The child’s profile is stored in session state once, then reused across follow-up questions in the same session.

> This is educational/supportive content and **not medical advice**.

---

## Inputs (what you must provide)
### 1) API keys (environment variables / `.env`)
The notebook initializes clients for:
- `GOOGLE_API_KEY` (Gemini via `google.genai` / ADK Gemini wrapper)
- `OPENAI_API_KEY` (GPT via `openai`)
- `ANTHROPIC_API_KEY` (Claude via `anthropic`)
It loads keys via `python-dotenv`.

### 2) Parent prompts (runtime)
You interact through:
- `await ask("My child's name is ... age ... diagnosis ...")`  → stores profile
- `await ask("What therapies do you recommend for ...?")`      → triggers research + summary
- `await ask("More fine-motor activities?")`                   → follow-up using saved profile

---

## What it does (process overview)
### A) Child profile memory (stateful)
- A **Profile Agent** extracts **name / age / diagnosis** from the parent message.
- It calls a `save_child_profile` tool to persist the profile in ADK **session state**.
- A `retrieve_child_profile` tool is used by downstream agents/tools to ensure the profile exists.

### B) Parallel research (multi-model)
The notebook defines **model-specific research tools** that:
1) read the stored profile from session state  
2) craft an age- + diagnosis-specific prompt  
3) call the corresponding model (GPT / Claude / Gemini)  
4) return structured research notes (or an error if profile is missing)

A **ParallelAgent** runs these researchers concurrently so you get broad coverage quickly.

### C) Aggregation (single final response)
An **Aggregator Agent** merges outputs from the parallel researchers into one coherent deliverable:
- organized therapy recommendations (often by domain: gross motor, fine motor, speech, OT/PT, routines, etc.)
- practical at-home suggestions
- safety/disclaimer language
- a single “FINAL SUMMARY” returned via the runner state

---

## Outputs (what you get)
### Console output
The helper `ask()` runs the ADK `runner.run_debug(...)` and prints:
- `FINAL SUMMARY` (the aggregator’s merged answer)

### Session state artifacts (in-memory / persistent depending on session service)
Within a single `user_id` + `session_id`, the notebook stores:
- `child_profile` (name, age, diagnosis)
- the aggregator’s `final_summary`
- (optionally) intermediate research outputs, depending on how tools write state

> If you restart the kernel or change session ids, the profile may be lost unless you’re using a database-backed session service.

---

## How to run
1) Run cells top-to-bottom to load keys, define tools/agents, and initialize the ADK `Runner`.
2) Start with a profile message (required once per session).
3) Ask therapy questions; the notebook will reuse the stored profile automatically.

---

## Limitations / known constraints
- If the **profile isn’t stored**, research tools return an error asking for name/age/diagnosis first.
- Output quality varies by **model availability** and API access.
- This notebook is a **prototype research/summarization pipeline**, not a clinical decision system.


In [ ]:
import openai
import anthropic
import google.genai
from google.adk import agents
print("All good!")

In [2]:
# Authentication & API clients
import os
from dotenv import load_dotenv

# Load variables from .env in the project root
load_dotenv()

# Tell google-genai to use the direct API, not Vertex AI
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"

# Optional: read API keys from environment (for sanity checks / debugging)
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

# Initialize OpenAI client
from openai import OpenAI
openai_client = OpenAI(api_key=OPENAI_API_KEY)

# Initialize Anthropic client
from anthropic import Anthropic
anthropic_client = Anthropic(api_key=ANTHROPIC_API_KEY)

# Safe, short debug prints (won't crash if keys are missing)
def _short(key: str | None) -> str:
    if not key:
        return "MISSING"
    return key[:5]

print("All API clients initialized successfully.")
print("Google:", _short(GOOGLE_API_KEY))
print("OpenAI:", _short(OPENAI_API_KEY))
print("Anthropic:", _short(ANTHROPIC_API_KEY))


All API clients initialized successfully.
Google: AIzaS
OpenAI: sk-pr
Anthropic: sk-an


In [3]:
# Core imports for Genex multi-agent + memory system

# ADK agents and orchestration
from google.adk.agents import Agent, LlmAgent, SequentialAgent, ParallelAgent

# Google LLM wrapper
from google.adk.models.google_llm import Gemini

# Runners and session services
from google.adk.runners import InMemoryRunner, Runner
from google.adk.sessions import DatabaseSessionService, InMemorySessionService

# Tools + tool context (needed for memory tools)
from google.adk.tools import FunctionTool
from google.adk.tools.tool_context import ToolContext

# App wrapper & compaction config (so sessions persist + are summarized)
from google.adk.apps.app import App, EventsCompactionConfig

# Low-level Google genai types
from google.genai import types

# Standard library
from typing import Any, Dict
import uuid

from typing import Any, Dict


In [4]:
# Retry configuration for all Gemini calls (used inside LlmAgent)
retry_config = types.HttpRetryOptions(
    attempts=5,                 # Max retry attempts
    exp_base=7,                 # Exponential backoff base
    initial_delay=1,            # Delay before 1st retry
    http_status_codes=[429, 500, 503, 504],  # Retry on rate-limit + server errors
)


### Helper Function Save and Retrieve Child's Profile

In [5]:
# CHILD PROFILE MEMORY TOOLS

def save_child_profile(
    tool_context: ToolContext,
    name: str,
    age_years: int,
    diagnosis: str,
) -> Dict[str, Any]:
    """
    Save the child's profile (e.g., Emma, 3, Down syndrome) into session state.

    This is scoped as user-level state, so it can be reused across turns
    in the same app/user/session.
    """
    # Normalize inputs a bit
    clean_name = name.strip()
    clean_diagnosis = diagnosis.strip()

    # Use 'user:child:*' prefix for good namespacing
    tool_context.state["user:child:name"] = clean_name
    tool_context.state["user:child:age_years"] = int(age_years)
    tool_context.state["user:child:diagnosis"] = clean_diagnosis

    return {
        "status": "success",
        "message": f"Stored child profile: {clean_name}, {age_years} years old, diagnosis: {clean_diagnosis}.",
        "name": clean_name,
        "age_years": int(age_years),
        "diagnosis": clean_diagnosis,
    }


def retrieve_child_profile(tool_context: ToolContext) -> Dict[str, Any]:
    """
    Retrieve the child's profile from session state.

    Returns:
        {
          "status": "success",
          "name": ...,
          "age_years": ...,
          "diagnosis": ...
        }
        or
        {
          "status": "error",
          "message": "..."
        }
    """
    name = tool_context.state.get("user:child:name")
    age_years = tool_context.state.get("user:child:age_years")
    diagnosis = tool_context.state.get("user:child:diagnosis")

    if name is None or age_years is None or diagnosis is None:
        return {
            "status": "error",
            "message": "Child profile is missing or incomplete. Please provide name, age, and diagnosis.",
        }

    return {
        "status": "success",
        "name": name,
        "age_years": age_years,
        "diagnosis": diagnosis,
    }


# Wrap as ADK tools so agents can call them
save_child_profile_tool = FunctionTool(func=save_child_profile)
retrieve_child_profile_tool = FunctionTool(func=retrieve_child_profile)

print("Child profile memory tools initialized.")


Child profile memory tools initialized.


### Profile agent

Ensures Emma is in state at the start of the pipeline. We keep a profile agent that stores Emma:

In [6]:
profile_agent = LlmAgent(
    name="profile_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    description="Collects and stores child profile info (name, age, diagnosis) in session state.",
    instruction="""
You are a profile manager for a pediatric support system.

Your responsibilities:

1. When the user describes their child (name, age, and diagnosis/condition),
   you MUST call the `save_child_profile` tool to store those details in session state.

2. If the user updates the profile (e.g., new age, additional diagnoses),
   call `save_child_profile` again with the updated information.

3. After saving, briefly confirm what you stored, for example:
   "I've saved Emma's profile: 3 years old with Down syndrome."

4. You MAY use `retrieve_child_profile` when you need to:
   - check what is currently stored, or
   - confirm the existing profile back to the user.

5. You MUST NOT:
   - give therapy recommendations,
   - discuss interventions,
   - or provide developmental advice.
   Your only job is to manage and confirm the stored profile.

All therapy suggestions will be handled by other specialist agents that use this stored profile.
""",
    tools=[save_child_profile_tool, retrieve_child_profile_tool],
)


### GPT Research Tool 

In [7]:
def gpt_child_research_tool(tool_context: ToolContext) -> Dict[str, Any]:
    """
    Use the stored child profile (name, age, diagnosis) to call GPT-4o-mini
    and get age- & diagnosis-specific therapy recommendations for that child.
    """
    # 1) Read Emma's (or any child's) profile from session state
    profile = retrieve_child_profile(tool_context)

    if profile.get("status") != "success":
        # Propagate a structured error for the LlmAgent to explain to the user
        return {
            "status": "error",
            "error_message": "Child profile is missing or incomplete. "
                             "Please tell me your child's name, age, and diagnosis first.",
        }

    name = profile["name"]
    age = profile["age_years"]
    diagnosis = profile["diagnosis"]

    # 2) Build a GPT prompt that is explicitly child- and age-specific
    prompt = f"""
You are a pediatric developmental specialist.

Child profile:
- Name: {name}
- Age: {age} years
- Diagnosis: {diagnosis}

Tasks:
1. Give a one-sentence summary of {diagnosis} in the context of a {age}-year-old child.
2. Provide recommended therapies tailored SPECIFICALLY for this child's age and condition,
   organized under:
   - language/communication
   - physical/movement
   - occupational/fine motor
   - social/emotional
   - cognitive/learning
3. Phrase everything as supportive, practical advice to the parents of {name}.
"""

    try:
        result = openai_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
        )
        content = result.choices[0].message.content
    except Exception as e:
        # Fail gracefully but explicitly so the orchestrator can react
        return {
            "status": "error",
            "error_message": f"GPT call failed: {e}",
        }

    # 3) Return a structured payload for the orchestrating agent
    return {
        "status": "success",
        "gpt_research": content,
        "child_profile_used": {
            "name": name,
            "age_years": age,
            "diagnosis": diagnosis,
        },
    }


# Wrap as ADK tool so LlmAgent can call it
gpt_child_tool = FunctionTool(func=gpt_child_research_tool)

print("GPT child research tool initialized.")


GPT child research tool initialized.


### Claude Research Tool

In [8]:
def claude_child_research_tool(tool_context: ToolContext) -> Dict[str, Any]:
    """
    Use the stored child profile (name, age, diagnosis) to call Claude
    and get age- & diagnosis-specific therapy recommendations.
    """
    # 1) Read child profile from session state
    profile = retrieve_child_profile(tool_context)

    if profile.get("status") != "success":
        return {
            "status": "error",
            "error_message": (
                "Child profile is missing or incomplete. "
                "Please tell me your child's name, age, and diagnosis first."
            ),
        }

    name = profile["name"]
    age = profile["age_years"]
    diagnosis = profile["diagnosis"]

    # 2) Build a Claude prompt tailored to this child
    prompt = f"""
You are a pediatric developmental specialist.

Child profile:
- Name: {name}
- Age: {age} years
- Diagnosis: {diagnosis}

Tasks:
1. Give a one-sentence summary of {diagnosis} in the context of a {age}-year-old child.
2. Provide recommended therapies tailored SPECIFICALLY for this child's age and condition,
   organized under:
   - language/communication
   - physical/movement
   - occupational/fine motor
   - social/emotional
   - cognitive/learning
3. Phrase everything as supportive, concrete advice to the parents of {name}.
"""

    try:
        response = anthropic_client.messages.create(
            model="claude-3-5-haiku-20241022",
            max_tokens=700,
            messages=[{"role": "user", "content": prompt}],
        )

        # Claude returns a list of content blocks; extract text safely
        text_blocks = [
            block.text
            for block in response.content
            if getattr(block, "type", None) == "text"
        ]
        claude_text = "\n".join(text_blocks).strip()

        if not claude_text:
            return {
                "status": "error",
                "error_message": "Claude returned no text content.",
            }

    except Exception as e:
        return {
            "status": "error",
            "error_message": f"Claude call failed: {e}",
        }

    # 3) Return structured result for the orchestrator / aggregator
    return {
        "status": "success",
        "claude_research": claude_text,
        "child_profile_used": {
            "name": name,
            "age_years": age,
            "diagnosis": diagnosis,
        },
    }


# Wrap this as an ADK FunctionTool so LlmAgents can call it
claude_child_tool = FunctionTool(func=claude_child_research_tool)

print("Claude child research tool initialized.")

Claude child research tool initialized.


### GPT Research Agent

In [9]:
gpt_researcher = LlmAgent(
    name="gpt_researcher",
    model=Gemini(
        model="gemini-2.5-flash",
        retry_options=retry_config
    ),
    description="Researcher agent that personalizes therapy insights using stored child profile.",
    instruction="""
    You are a pediatric development research agent.
    You MUST always call the tool `gpt_child_tool`.

    Do NOT generate your own text.
    Do NOT summarize or analyze on your own.

    Your ONLY job:
    - Retrieve the child's profile using the tool_context
    - Call gpt_child_research_tool
    - Return the tool output as your final answer
    """,
    tools=[gpt_child_tool],
    output_key="gpt_research",
)

### Claude Research Agent 

In [10]:
claude_researcher = LlmAgent(
    name="claude_researcher",
    model=Gemini(
        model="gemini-2.5-flash",
        retry_options=retry_config,
    ),
    description="Researcher agent that personalizes therapy insights using Claude and the stored child profile.",
    instruction="""
    You are a pediatric development research agent that delegates all content generation to Claude.

    You MUST always call the tool `claude_child_tool`.

    Rules:
    - Do NOT generate your own domain answer.
    - Do NOT summarize or rephrase on your own.
    - Do NOT answer directly from your own knowledge.
    - Your ONLY job is:
        1) Use the tool (which already uses the child profile from session state),
        2) Return the tool's output as your final answer.
    If the tool returns status="error", explain the error_message to the user
    and suggest that they provide the child's name, age, and diagnosis.
    """,
    tools=[claude_child_tool],
    output_key="claude_research",
)


### Gemini Research Agent 

In [11]:
# Wrap retrieve_child_profile as a tool
retrieve_child_profile_tool = FunctionTool(func=retrieve_child_profile)

gemini_researcher = LlmAgent(
    name="gemini_researcher",
    model=Gemini(
        model="gemini-2.5-flash",
        retry_options=retry_config,
    ),
    description="Gemini researcher that personalizes therapy recommendations using saved child profile.",
    instruction="""
You are a pediatric developmental specialist and part of a multi-model research team.

Your REQUIRED behavior:

1. Always call the tool `retrieve_child_profile` to retrieve:
   - child's name
   - child's age (years)
   - child's diagnosis/condition

2. If the tool returns an error (profile missing):
   - Do NOT invent an answer.
   - Say: "I cannot provide recommendations because no child profile is stored yet.
     Please provide your child's name, age, and diagnosis."

3. If the profile exists, produce a personalized research summary:
   - Begin with: "For [NAME], [AGE] years old with [DIAGNOSIS], ..."
   - Provide **therapy recommendations tailored to both age & diagnosis**.
   - Organize the recommendations under the exact sections:
        • language & communication
        • physical & movement
        • occupational & fine-motor
        • social & emotional
        • cognitive & learning

4. The tone must be warm, supportive, and parent-focused.
5. Keep the response *concise but specific*.
6. Do NOT consult external tools or call any API — only use the profile and your own reasoning.

Your answer will be stored under the key `gemini_research` and later used by an aggregator.
""",
    tools=[retrieve_child_profile_tool],
    output_key="gemini_research",
)


### Root pipeline
#### profile → parallel research → aggregator -> root agent

In [12]:
# pipeline: Parallel Agent
parallel_research_team = ParallelAgent(
    name="parallel_research_team",
    sub_agents=[gemini_researcher, gpt_researcher, claude_researcher],
)

# pipeline: Aggregator Agent
aggregator_agent = LlmAgent(
    name="aggregator_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    tools=[retrieve_child_profile_tool],
    description="Final summarizer that merges Gemini, GPT, and Claude research into one parent-friendly plan.",
    instruction="""
You are the final summarizer in a multi-model research system.

Upstream agents have already run and stored their outputs in state under these keys
(if they succeeded):

- 'gemini_research'
- 'gpt_research'
- 'claude_research'

You MUST NOT try to reference them as {gemini_research} or similar; instead, you
should conceptually treat them as background sources that the system has already
given you.

Steps:

1. Call `retrieve_child_profile` to get:
   - child name
   - age in years
   - diagnosis

2. If the profile tool returns an error (no child info stored):
   - Do NOT try to summarize.
   - Respond with a short, clear message:
     "I can't summarize therapies yet because no child profile is stored.
      Please tell me your child's name, age, and diagnosis."

3. If the profile exists, produce your answer in EXACTLY this structure:

   Line 1:
     "[NAME] is [AGE] years old and has [DIAGNOSIS]."

   Line 2:
     " [DIAGNOSIS] is ..."
     → one concise sentence explaining the condition in plain language.

   Then a section:

   "Therapy Recommendations by Category"

   Under this, create the following subsections, each with ONE short paragraph
   that summarizes what the three researchers broadly agree on:

   - **Language & Communication**
   - **Physical & Movement**
   - **Occupational & Fine Motor**
   - **Social & Emotional**
   - **Cognitive & Learning**

   Use the information from whatever research keys are present in state
   (gemini_research, gpt_research, claude_research). If one of them is missing,
   just ignore it and rely on the others.

4. After that, add a final section:

   "Differences Between Researchers"

   - If they largely agree overall:
       Write one short sentence stating that all sources are broadly aligned.
   - If there are meaningful differences (e.g., one emphasizes professional services
     more while another emphasizes home play-based work), list 2–4 bullet points
     describing the key differences.

Tone rules:
- Warm, supportive, and parent-friendly.
- Concise but specific.
- No long bullet lists; keep it readable.
""",
    output_key="final_summary",
)

# Pipeline: Root Agent
root_agent = SequentialAgent(
    name="research_system",
    sub_agents=[
        profile_agent,          # reads or updates the child profile in state
        parallel_research_team, # runs Gemini, GPT, Claude in parallel
        aggregator_agent,       # produces the final summary only
        ],
)

### Persistent sessions per child / family

In [13]:
# App & persistence configuration
APP_NAME = "genex_research_app"
USER_ID = "emma_parents"

# Simple in-memory session service for development
session_service = InMemorySessionService()

# Define the Genex research app with event compaction
research_app = App(
    name=APP_NAME,
    root_agent=root_agent,   # SequentialAgent: profile → parallel → aggregator
)

# Runner that ties everything together
runner = Runner(
    app=research_app,
    session_service=session_service,
)

print("Genex research app initialized with persistent sessions.")
print(f"   - App name: {APP_NAME}")

Genex research app initialized with persistent sessions.
   - App name: genex_research_app


In [14]:
# First run (once per session)
await runner.run_debug(
    "Emma. 3 years old, Down syndrome.",
    user_id="emma_parents",
    session_id="emma-session-001",
    quiet=True,
    verbose=False,
)

_ResourceExhaustedError: 
On how to mitigate this issue, please refer to:

https://google.github.io/adk-docs/agents/models/#error-code-429-resource_exhausted


429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash-lite\nPlease retry in 27.173882984s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_co

In [15]:
# We can do this instead for the first time running to inspect the final output

# First-time run: store Emma's profile + generate full research summary
response_events = await runner.run_debug(
    "Emma. 3 years old, Down syndrome.",
    user_id="emma_parents",        # must match APP_NAME/USER_ID block
    session_id="emma-session-001", # your first persistent session
    quiet=True,                    # prevent noisy output
    verbose=False
)

# Extract the last event — aggregator output
final_event = response_events[-1]

# Check whether aggregator emitted a final summary
if hasattr(final_event.actions, "state_delta") and "final_summary" in final_event.actions.state_delta:
    final_summary = final_event.actions.state_delta["final_summary"]
    print("\n FINAL SUMMARY (Aggregator)\n")
    print(final_summary)
else:
    print("No final summary produced. Check if aggregator was reached.")

_ResourceExhaustedError: 
On how to mitigate this issue, please refer to:

https://google.github.io/adk-docs/agents/models/#error-code-429-resource_exhausted


429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash-lite\nPlease retry in 22.749940455s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_co

### Mental model to remember

One session = one child context

First message must include name + age + diagnosis

After that, never repeat it unless it changes

Everything else builds on top

In [ ]:
# Define a helper
# The helper defines a tiny wrapper so we can type specific questions with await ask
# instead of instead of rewriting the full runner call every time.

async def ask(prompt: str):
    events = await runner.run_debug(
        prompt,
        user_id="emma_parents",
        session_id="emma-session-001",
        quiet=True,
        verbose=False,
    )

    # last event is usually aggregator output
    final_event = events[-1]
    summary = final_event.actions.state_delta.get("final_summary")

    print("\nFINAL SUMMARY\n")
    print(summary)

In [ ]:
await ask("My child's name is Emma. She is 3 years old and has Down syndrome.")

In [ ]:
await ask("What therapies do you recommend for Emma's overall development?")

In [ ]:
await ask("Can you suggest more fine-motor activities for Emma?")

### Summary of the framework

[Profile Agent] → stores child:name, age, diagnosis

[Parallel Research Team]
   - ├── Gemini researcher → Gemini_child_tool → personalized research
   - ├── GPT researcher    → GPT_child_tool    → personalized research
   - └── Claude researcher → Claude_child_tool → personalized research

[Aggregator Agent] → merges 3 sources using child profile


### Summary of the Genex Research Framework (Current State)

This framework is a **stateful, multi-agent, multi-model research system** designed to generate **child-specific developmental guidance** in a structured, extensible, and safe way. It cleanly separates *data collection*, *expert research*, and *synthesis*, making it easy to extend later with milestone tracking, developmental-age estimation, and activity planning.

---

#### 1. Profile Agent — Child Context & Memory
**Role:** Establishes and maintains persistent child context across turns.

- Extracts the child’s **name, age, and diagnosis/condition** from user messages.
- Stores this information in **session state** (memory), scoped to `(user_id, session_id)`.
- Enables all downstream agents to operate on a **consistent, shared child profile**.
- Does **not** provide therapy guidance or reasoning — its sole responsibility is identity and memory management.

> Output:  
> `state["user:child:name"]`, `state["user:child:age_years"]`, `state["user:child:diagnosis"]`

---

#### 2. Parallel Research Team — Independent Expert Perspectives
**Role:** Gather diverse, independent research perspectives tailored to the child.

A `ParallelAgent` runs three specialized researcher agents concurrently:

- **Gemini Researcher**
  - Retrieves the stored child profile.
  - Delegates content generation to Gemini via a tool.
  - Produces structured, age- and diagnosis-specific developmental insights.

- **GPT Researcher**
  - Uses the same stored child profile.
  - Calls GPT via a tool for practical, applied recommendations.
  - Returns child-specific research output.

- **Claude Researcher**
  - Also retrieves the child profile.
  - Delegates reasoning and narrative generation to Claude.
  - Emphasizes clarity, tone, and developmental nuance.

Key characteristics:
- Researchers **do not communicate with each other**.
- No summarization or merging happens at this stage.
- Each model’s bias and strengths are preserved intentionally.
- All outputs are written to shared state under distinct keys.

> Outputs:  
> `state["gemini_research"]`  
> `state["gpt_research"]`  
> `state["claude_research"]`

---

#### 3. Aggregator Agent — Synthesis & Parent-Facing Output
**Role:** Convert multi-model research into one coherent, supportive answer.

- Re-retrieves the child profile to ground the final message in context.
- Reads all available researcher outputs from state.
- Synthesizes them into a **single, structured, parent-friendly summary**.
- Enforces a consistent format, tone, and level of detail.
- Highlights areas of agreement and notes meaningful differences between models.

The aggregator is deliberately separated from research so that:
- Summarization logic can evolve independently.
- Additional research sources can be added later.
- Output format remains stable even if models change.

> Output:  
> `state["final_summary"]`

---

#### Overall Data & Control Flow

- User input
- ↓
- [Profile Agent] → child profile stored in session state
- ↓
- [Parallel Research Team]
- ├─ Gemini researcher → gemini_research
- ├─ GPT researcher → gpt_research
- └─ Claude researcher → claude_research
- ↓
- [Aggregator Agent]
- → final_summary (parent-facing response)

---

#### Why This Architecture Matters

- **Personalized:** Every response is tied to a specific child, not a generic condition.
- **Robust:** Multiple LLMs reduce single-model bias or failure modes.
- **Explainable:** Intermediate research outputs are preserved in state.
- **Extensible:** Future pipelines (CDC milestones, developmental age, activity planning) can reuse the same profile memory and aggregation patterns.
- **Production-ready:** Clear separation of concerns supports testing, iteration, and scaling.

This framework forms the **foundational research layer** of Genex, upon which milestone assessment, therapy planning, and long-term progress tracking will be built.